In [121]:
# Import Libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import random


### Get Data from Kaggle

In [122]:
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [123]:
# !kaggle datasets download -d jeffgallini/college-football-team-stats-2019 --unzip

### Import Kaggle Data into Notebook

In [124]:
# df = pd.read_csv('./cfb23.csv')
# df

### Webscrape Game Data

In [125]:
# Create a list of seasons 2014-2023
seasons = list(range(2014, 2025))
seasons

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [126]:
# Create an empty list to store DataFrames for each season
dfs_per_season = []

# Iterate through seasons
for season in seasons:
    # Set URL
    url =  f'https://www.sports-reference.com/cfb/years/{season}-schedule.html'
    print(url)

    # Get game results ('id': 'schedule' is an HTML attribute on the website)
    results_df = pd.read_html(url, header=1, attrs={'id': 'schedule'})[0]

    # Append the dataframe to the list
    dfs_per_season.append(results_df)

    # Pause the program so don't get flagged by website
    time.sleep(random.randint(4,5))

dfs_per_season

https://www.sports-reference.com/cfb/years/2014-schedule.html
https://www.sports-reference.com/cfb/years/2015-schedule.html
https://www.sports-reference.com/cfb/years/2016-schedule.html
https://www.sports-reference.com/cfb/years/2017-schedule.html
https://www.sports-reference.com/cfb/years/2018-schedule.html
https://www.sports-reference.com/cfb/years/2019-schedule.html
https://www.sports-reference.com/cfb/years/2020-schedule.html
https://www.sports-reference.com/cfb/years/2021-schedule.html
https://www.sports-reference.com/cfb/years/2022-schedule.html
https://www.sports-reference.com/cfb/years/2023-schedule.html
https://www.sports-reference.com/cfb/years/2024-schedule.html


[       1 1.1  Aug 27, 2014   7:00 PM  Wed       Georgia State   38 Unnamed: 7  \
 0      2   1  Aug 28, 2014   7:00 PM  Thu               Akron   41        NaN   
 1      3   1  Aug 28, 2014  10:30 PM  Thu  (19) Arizona State   45        NaN   
 2      4   1  Aug 28, 2014   7:00 PM  Thu    Central Michigan   20        NaN   
 3      5   1  Aug 28, 2014   7:02 PM  Thu    Louisiana-Monroe   17        NaN   
 4      6   1  Aug 28, 2014   7:00 PM  Thu           Minnesota   42        NaN   
 ..   ...  ..           ...       ...  ...                 ...  ...        ...   
 882  865  20   Jan 2, 2015   6:45 PM  Fri           (14) UCLA   40          N   
 883  866  20   Jan 3, 2015  12:00 PM  Sat             Florida   28          N   
 884  867  20   Jan 4, 2015   9:00 PM  Sun              Toledo   63          N   
 885   Rk  Wk          Date      Time  Day              Winner  Pts        NaN   
 886  868  21  Jan 12, 2015   8:30 PM  Mon      (5) Ohio State   42          N   
 
      Abilene 

In [127]:
# Look at seasons (adjust index to see specific season)
df_season = dfs_per_season[5]
df_season.head()

,1,1.1,"Aug 24, 2019",Unnamed: 3,Sat,(8) Florida,24,N,Miami (FL),20,"Camping World Stadium - Orlando, Florida"
0,2,1,"Aug 24, 2019",NaN,Sat,Hawaii,45,NaN,Arizona,38,"Aloha Stadium - Honolulu, Hawaii"
1,Rk,Wk,Date,Time,Day,Winner,Pts,NaN,Loser,Pts,Notes
2,3,2,"Aug 29, 2019",NaN,Thu,Alabama-Birmingham,24,NaN,Alabama State,19,"Legion Field - Birmingham, Alabama"
3,4,2,"Aug 29, 2019",NaN,Thu,Arizona State,30,NaN,Kent State,7,"Sun Devil Stadium - Tempe, Arizona"
4,5,2,"Aug 29, 2019",NaN,Thu,Bowling Green,46,NaN,Morgan State,3,"Doyt Perry Stadium - Bowling Green, Ohio"


In [128]:
# Make a copy
scraped_df = dfs_per_season

In [129]:
# Clean data for each DataFrame using loop
for i, season_df in enumerate(scraped_df):
    # Add columns
    season_df.columns = ['Rk', 'Wk', 'Date', 'Time', 'Day', 'Winner', 'Winner Pts', '?', 'Loser', 'Loser Pts', 'Notes']

    # Drop unnecssary columns
    season_df.drop(['Rk', 'Day', '?', 'Notes'], axis=1, inplace=True)

    # Convert 'Winner Pts' and 'Loser Pts' to numeric
    season_df.loc[:, 'Winner'] = season_df['Winner'].str.replace(r'\(\d+\)\s*', '', regex=True)
    season_df.loc[:, 'Loser'] = season_df['Loser'].str.replace(r'\(\d+\)\s*', '', regex=True)

    # Find rows with NaN values (these are from the original dataset so take them out)
    rows_with_nan = season_df[season_df.isna().any(axis=1)]

    # Remove rows with NaN values
    season_df = season_df.dropna()

    # Standardize the winner and loser names (remove rankings)
    season_df['Winner'] = season_df['Winner'].str.replace(r'\(\d+\)\s*', '')
    season_df['Loser'] = season_df['Loser'].str.replace(r'\(\d+\)\s*', '')

    # Insert Season into the dataframe
    season_df.insert(loc=0, column='Season', value=seasons[i])
    dfs_per_season[i] = season_df

    # Drop rows with NaN values
    season_df = season_df.dropna(subset=['Date', 'Time', 'Winner', 'Loser'])

    # Combine the date and time columns into a single column 'DateTime'
    season_df['DateTime'] = pd.to_datetime(season_df['Date'] + ' ' + season_df['Time'], errors='coerce')

    # Drop rows with NaT in 'DateTime'
    season_df = season_df.dropna(subset=['DateTime'])

    # Drop original 'Date' and 'Time' columns
    season_df.drop(['Date', 'Time'], axis=1, inplace=True)

    # Update the dataframe in the list
    dfs_per_season[i] = season_df

/var/folders/zm/fr635kq51d35fklpj2hv3s240000gn/T/ipykernel_46813/2381638167.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_df['Winner'] = season_df['Winner'].str.replace(r'\(\d+\)\s*', '')
/var/folders/zm/fr635kq51d35fklpj2hv3s240000gn/T/ipykernel_46813/2381638167.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_df['Loser'] = season_df['Loser'].str.replace(r'\(\d+\)\s*', '')
/var/folders/zm/fr635kq51d35fklpj2hv3s240000gn/T/ipykernel_46813/2381638167.py:31: UserWarning: Could not infe

In [134]:
# Concatenate dataframes
combined_df = pd.concat(dfs_per_season, ignore_index=True)
combined_df

,Season,Wk,Winner,Winner Pts,Loser,Loser Pts,DateTime
0,2014,1,Akron,41,Howard,0,2014-08-28 19:00:00
1,2014,1,Arizona State,45,Weber State,14,2014-08-28 22:30:00
2,2014,1,Central Michigan,20,Chattanooga,16,2014-08-28 19:00:00
3,2014,1,Louisiana-Monroe,17,Wake Forest,10,2014-08-28 19:02:00
4,2014,1,Minnesota,42,Eastern Illinois,20,2014-08-28 19:00:00
...,...,...,...,...,...,...,...
8261,2024,14,Texas Tech,56,Oklahoma State,48,2024-11-23 15:30:00
8262,2024,14,Utah State,41,San Diego State,20,2024-11-23 15:30:00
8263,2024,14,West Virginia,31,Central Florida,21,2024-11-23 15:30:00
8264,2024,15,Akron,21,Toledo,14,2024-11-26 19:00:00


In [131]:
# Duplicate the Dataframe
df_duplicated = combined_df.copy()

# Swap the information between 'Winner' and 'Loser' columns in the duplicated dataframe
df_duplicated[['Winner', 'Winner Pts', 'Loser', 'Loser Pts']] = combined_df[['Loser', 'Loser Pts', 'Winner', 'Winner Pts']]

# Add a new column 'Win' to indicate if team won (true) or lost (false)
df_duplicated['Win'] = False
combined_df['Win'] = True

# Concatenate the original and duplicated DataFrames
reshaped_df = pd.concat([combined_df, df_duplicated], ignore_index=True)

# Rename columns
reshaped_df = reshaped_df.rename(columns={'Winner': 'Team',
                                        'Winner Pts': 'Points Scored', 
                                        'Loser': 'Opponent', 
                                        'Loser Pts': 'Points Allowed'})

reshaped_df

,Season,Wk,Team,Points Scored,Opponent,Points Allowed,DateTime,Win
0,2014,1,Akron,41,Howard,0,2014-08-28 19:00:00,True
1,2014,1,Arizona State,45,Weber State,14,2014-08-28 22:30:00,True
2,2014,1,Central Michigan,20,Chattanooga,16,2014-08-28 19:00:00,True
3,2014,1,Louisiana-Monroe,17,Wake Forest,10,2014-08-28 19:02:00,True
4,2014,1,Minnesota,42,Eastern Illinois,20,2014-08-28 19:00:00,True
...,...,...,...,...,...,...,...,...
16527,2024,14,Oklahoma State,48,Texas Tech,56,2024-11-23 15:30:00,False
16528,2024,14,San Diego State,20,Utah State,41,2024-11-23 15:30:00,False
16529,2024,14,Central Florida,21,West Virginia,31,2024-11-23 15:30:00,False
16530,2024,15,Toledo,14,Akron,21,2024-11-26 19:00:00,False


In [132]:
# Drop problematic rows
reshaped_df = reshaped_df[
    (reshaped_df['Season'] != 'Wk') &
    (reshaped_df['Team'] != 'Loser') &
    (reshaped_df['Opponent'] != 'Pts')
]

reshaped_df.head(50)

,Season,Wk,Team,Points Scored,Opponent,Points Allowed,DateTime,Win
0,2014,1,Akron,41,Howard,0,2014-08-28 19:00:00,True
1,2014,1,Arizona State,45,Weber State,14,2014-08-28 22:30:00,True
2,2014,1,Central Michigan,20,Chattanooga,16,2014-08-28 19:00:00,True
3,2014,1,Louisiana-Monroe,17,Wake Forest,10,2014-08-28 19:02:00,True
4,2014,1,Minnesota,42,Eastern Illinois,20,2014-08-28 19:00:00,True
5,2014,1,Mississippi,35,Boise State,13,2014-08-28 20:00:00,True
6,2014,1,New Mexico State,28,Cal Poly,10,2014-08-28 20:00:00,True
7,2014,1,Northern Illinois,55,Presbyterian,3,2014-08-28 19:00:00,True
8,2014,1,Rutgers,41,Washington State,38,2014-08-28 22:00:00,True
9,2014,1,San Jose State,42,North Dakota,10,2014-08-28 22:00:00,True


In [136]:
# Sort data by DateTime so everything is synchronized
df = reshaped_df.sort_values("DateTime")
df = df.reset_index(drop=True)
df

,Season,Wk,Team,Points Scored,Opponent,Points Allowed,DateTime,Win
0,2014,1,South Carolina,28,Texas A&M,52,2014-08-28 18:00:00,False
1,2014,1,Texas A&M,52,South Carolina,28,2014-08-28 18:00:00,True
2,2014,1,Akron,41,Howard,0,2014-08-28 19:00:00,True
3,2014,1,Presbyterian,3,Northern Illinois,55,2014-08-28 19:00:00,False
4,2014,1,Eastern Illinois,20,Minnesota,42,2014-08-28 19:00:00,False
...,...,...,...,...,...,...,...,...
16527,2024,14,Air Force,22,Nevada,19,2024-11-23 22:30:00,True
16528,2024,15,Buffalo,43,Kent State,7,2024-11-26 19:00:00,True
16529,2024,15,Akron,21,Toledo,14,2024-11-26 19:00:00,True
16530,2024,15,Toledo,14,Akron,21,2024-11-26 19:00:00,False


In [137]:
# Push to CSV file
df.to_csv('./game_results.csv')

In [85]:
teams = df['Team'].unique()
len(teams)

248

In [86]:
print(list(range(14,25)))

[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [146]:
# create column headers for final df
col = ['Game Number', 'Date', 'Location', 'Opponent', 'Result', 'Passing Completion', 
       'Passing Attempts', 'Passing Percentage', 'Passing Yards', 'Passing TDs',
         'RushingA ttempts', 'Rushing Yards', 'Rushing Average', 'Rushing TDs', 'Offensive Plays', 
         'Offensive Yards', 'Offensive Average', 'First Down Pass', 'First Down Rush', 
         'First Down Penalties', 'First Down Total', 'Penalties', 'Penalty Yards', 'Fumbles', 
         'Interceptions', 'Total Turnovers']

# Standardize team names for URL format
standardized_teams = [team.lower().replace(' ', '-').replace('&', 'and') for team in teams]

# Initialize final DataFrame
final_df = pd.DataFrame()

# Get length of standardized teams
num_teams = len(standardized_teams)
ticker = 0
# Loop over each team
for team in standardized_teams:
    ticker += 1
    print(f"Fetching data for team: {team}")
    print(f'{ticker} / {num_teams} teams scraped')
    team_data = pd.DataFrame()
    
    for year in range(2014, 2025):
        try:
            url = f'https://www.sports-reference.com/cfb/schools/{team}/{year}/gamelog/'
            
            # Fetch data
            season = pd.read_html(url, header=0)[0]
            
            # Clean and format data
            season = season.iloc[1:-1]  # Remove top and bottom summary rows
            season.columns = col  # Set column names
            season['Team'] = team
            season['Season'] = year
            
            # Append to team_data
            team_data = pd.concat([team_data, season], ignore_index=True)
            
            # Add a delay to avoid rate limits
            time.sleep(random.randint(4,5))
        
        except Exception as e:
            print(f"Failed to fetch data for {team} in {year}: {e}")
            continue
    
    # Append team_data to final_df
    final_df = pd.concat([final_df, team_data], ignore_index=True)
    
    # Append team_data to final_df
    final_df = pd.concat([final_df, team_data], ignore_index=True)

Fetching data for team: nevada
1 / 248 teams scraped
Failed to fetch data for nevada in 2014: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2015: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2016: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2017: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2018: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2019: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2020: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2021: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2022: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2023: HTTP Error 429: Too Many Requests
Failed to fetch data for nevada in 2024: HTTP Error 429: Too Many Requests
Fetching data for team: new-mexico
2 / 248 teams scraped
Failed to fetch data for new-mexico in 2014: HTTP Error 429: Too 

KeyboardInterrupt: 

In [141]:
final_df

,Game Number,Date,Location,Opponent,Result,Passing Completion,Passing Attempts,Passing Percentage,Passing Yards,Passing TDs,...,First Down Rush,First Down Penalties,First Down Total,Penalties,Penalty Yards,Fumbles,Interceptions,Total Turnovers,Team,Season
0,1,2014-08-30,NaN,Southern Utah,W (28-19),30,41,73.2,303,1,...,14,0,30,4,19,1,1,2,nevada,2014
1,2,2014-09-05,NaN,Washington State,W (24-13),12,21,57.1,110,0,...,10,3,17,9,90,0,0,0,nevada,2014
2,3,2014-09-13,@,Arizona,L (28-35),29,39,74.4,321,3,...,8,0,25,8,74,0,0,0,nevada,2014
3,4,2014-09-27,@,San Jose State,W (21-10),10,18,55.6,64,0,...,12,2,16,8,56,0,0,0,nevada,2014
4,5,2014-10-04,NaN,Boise State,L (46-51),23,43,53.5,306,3,...,6,3,24,3,22,0,4,4,nevada,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,8,2015-10-31,NaN,Tulsa,L (31-40),23,46,50.0,322,4,...,7,2,23,1,10,1,1,2,southern-methodist,2015
282,9,2015-11-06,NaN,Temple,L (40-60),18,30,60.0,212,1,...,6,4,20,5,53,1,1,2,southern-methodist,2015
283,10,2015-11-14,@,Navy,L (14-55),9,24,37.5,124,1,...,11,0,15,4,45,0,2,2,southern-methodist,2015
284,11,2015-11-21,NaN,Tulane,W (49-21),18,25,72.0,195,0,...,11,0,21,14,123,0,0,0,southern-methodist,2015
